In [13]:
from gensim.models import KeyedVectors

# Load GloVe embeddings in word2vec format
glove_path = "glove.840B.300d.word2vec.bin"
glove_model = KeyedVectors.load_word2vec_format(glove_path, binary=True, limit=500000)

In [14]:
import torch

vocab_size = len(glove_model.key_to_index)  # Number of words in GloVe
embedding_dim = glove_model.vector_size  # 300 in the case of GloVe 300d

# Create embedding matrix (vocab_size x embedding_dim)
embedding_matrix = torch.zeros((vocab_size, embedding_dim))

for i, word in enumerate(glove_model.key_to_index):
    embedding_matrix[i] = torch.tensor(glove_model[word])

# Use the embedding matrix with PyTorch nn.Embedding
glove_embeddings = torch.nn.Embedding.from_pretrained(embedding_matrix, freeze=False)  # Freeze=False if you want to fine-tune

In [5]:
glove_embeddings

Embedding(500000, 300)

In [9]:
from preprocess_data import load_data

train_df, _ = load_data()

Loading preprocessed Parquet files...... Done!


In [10]:
glove_vocab = {word: idx for idx, word in enumerate(glove_model.index_to_key)}

In [11]:
import nltk
nltk.download('punkt')  # Ensure you have the tokenizer data

from nltk.tokenize import word_tokenize

def glove_tokenize(text):
    tokens = word_tokenize(text.lower())  # Lowercasing to match GloVe case
    token_indices = [glove_vocab.get(token, glove_vocab.get('<unk>', 0)) for token in tokens]
    return token_indices

# Apply this to your dataset
train_df['Text_tokens_glove'] = train_df['Text'].apply(glove_tokenize)
train_df['Summary_tokens_glove'] = train_df['Summary'].apply(glove_tokenize)

[nltk_data] Downloading package punkt to /home/m33039/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


AttributeError: 'NoneType' object has no attribute 'lower'